### Importation packages

In [1]:
import numpy as np
import pandas as pd

### Jointures tables

In [8]:
annees = ['2019']
compet = ['cross', 'stade', 'salle']
data_tot = pd.DataFrame()

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data_tot = pd.concat([data_tot, data])    

In [9]:
annees = ['2020', '2021', '2022']
compet = ['cross', 'HS', 'salle', 'stade']

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data_tot = pd.concat([data_tot, data]) 



In [10]:
annees = ['2023']
compet = ['cross', 'salle']

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data_tot = pd.concat([data_tot, data])     

In [12]:
data_tot.to_csv(f'Resultats_totaux.csv') 

In [8]:
data_tot = pd.read_csv("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_totaux.csv")
data_tot.head()

C:\Users\alexa\AppData\Local\Temp\ipykernel_31784\2360565820.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_tot = pd.read_csv("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_totaux.csv")


,Unnamed: 0.1,Unnamed: 0,Position,Nom_athlete,Performance,Annee_naissance,Categorie,Club,Département,Ligue,Epreuve,Nom_competition,Lieu_competition,Date_competition,Heure_epreuve,Departement_competition,Region_competition,Label_competition
0,0,0,1,BRESCH Nathan,9'31'',6.0,MIM,Ibal,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label
1,1,1,2,BOYER Elliot,9'35'',5.0,MIM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label
2,2,2,3,GOETTEN Moritz (Ger),9'41'',7.0,BEM,Asv Landau,,,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label
3,3,3,4,LECHNER Arnaud,9'49'',7.0,BEM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label
4,4,4,5,BOSSUYT Thibault,9'56'',6.0,MIM,Ana Sg Wantzenau,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label


### Nettoyage base / retraitement données

In [9]:
data_tot.drop(data_tot.columns[[0,1,14]], axis=1, inplace=True)
data_tot.head()

,Position,Nom_athlete,Performance,Annee_naissance,Categorie,Club,Département,Ligue,Epreuve,Nom_competition,Lieu_competition,Date_competition,Departement_competition,Region_competition,Label_competition
0,1,BRESCH Nathan,9'31'',6.0,MIM,Ibal,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
1,2,BOYER Elliot,9'35'',5.0,MIM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
2,3,GOETTEN Moritz (Ger),9'41'',7.0,BEM,Asv Landau,,,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
3,4,LECHNER Arnaud,9'49'',7.0,BEM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
4,5,BOSSUYT Thibault,9'56'',6.0,MIM,Ana Sg Wantzenau,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label


In [23]:
data_tot["Annee_naissance"]=data_tot["Annee_naissance"].fillna(9999)
data_tot["Annee_naissance"]=data_tot["Annee_naissance"].astype(int)
data_tot["Annee_naissance"]=data_tot["Annee_naissance"].apply(lambda x: x + 2000 if 0 <= x <= 23 else (x + 1900 if 24 <= x <= 99 else x))

In [27]:
data_tot["Performance"]=data_tot["Performance"].apply(lambda x: np.nan if x == '-' else x)
data_tot.dropna(subset=["Performance"], inplace=True)

In [24]:
data_tot.head()


,Position,Nom_athlete,Performance,Annee_naissance,Categorie,Club,Département,Ligue,Epreuve,Nom_competition,Lieu_competition,Date_competition,Departement_competition,Region_competition,Label_competition
0,1,BRESCH Nathan,9'31'',2006,MIM,Ibal,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
1,2,BOYER Elliot,9'35'',2005,MIM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
2,3,GOETTEN Moritz (Ger),9'41'',2007,BEM,Asv Landau,,,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
3,4,LECHNER Arnaud,9'49'',2007,BEM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
4,5,BOSSUYT Thibault,9'56'',2006,MIM,Ana Sg Wantzenau,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,67,G-E,Pas de label
